In [ ]:
import re  # 정규 표현식 모듈 추가
import pandas as pd  # CSV 저장을 위한 pandas 추가
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException, TimeoutException, StaleElementReferenceException
import time

# ChromeDriver 옵션 설정
options = webdriver.ChromeOptions()
options.add_argument("--disable-gpu")
options.add_argument("--window-size=1920,1080")
options.add_experimental_option("excludeSwitches", ["enable-logging", "enable-automation"])
options.add_argument("--disable-blink-features=AutomationControlled")

# ChromeDriver 초기화
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)

# 네이버 지도 웹사이트로 이동 (테스트 URL)
driver.get('https://map.naver.com/p/search/%EB%B1%85%EB%B1%85%EB%A7%89%EA%B5%AD%EC%88%98/place/33964233?c=15.00,0,0,0,dh&placePath=%3Fentry%253Dbmp')

# 페이지 로드 대기
WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.ID, "entryIframe")))

try:
    # entryIframe이 로드될 때까지 대기 후 전환
    WebDriverWait(driver, 10).until(EC.frame_to_be_available_and_switch_to_it((By.ID, "entryIframe")))

    # 가게 정보 가져오기
    store_name_element = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.XPATH, "//*[@id='_title']/div/span[1]"))
    )
    store_name = store_name_element.text

    # 업종 카테고리 가져오기
    category_element = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.XPATH, "//*[@id='_title']/div/span[2]"))
    )
    category = category_element.text

    # 별점 가져오기 (부모 요소의 텍스트 가져오기)
    rating_element = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.XPATH, "/html/body/div[3]/div/div/div/div[2]/div[1]/div[2]/span[1]"))
    )
    rating_text = rating_element.text
    rating = rating_text.replace("별점", "").strip()

    # 방문자 리뷰 가져오기
    visitor_review_element = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.XPATH, "//*[@id='app-root']/div/div/div/div[2]/div[1]/div[2]/span[2]/a"))
    )
    visitor_review = visitor_review_element.text.split("리뷰")[1].strip()

    # 블로그 리뷰 가져오기
    blog_review_element = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.XPATH, "//*[@id='app-root']/div/div/div/div[2]/div[1]/div[2]/span[3]/a"))
    )
    blog_review = blog_review_element.text.split("리뷰")[1].strip()

    # 결과 출력
    print("가게 이름:", store_name)
    print("업종 카테고리:", category)
    print("별점:", rating)
    print("방문자 리뷰:", visitor_review)
    print("블로그 리뷰:", blog_review)

    # '리뷰' 탭 클릭하기
    try:
        review_tab_element = WebDriverWait(driver, 15).until(
            EC.element_to_be_clickable((By.XPATH, "//span[text()='리뷰']"))
        )
        review_tab_element.click()
        WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/h2/div[1]/em')))
        print("리뷰 탭 클릭 완료")
    except Exception as e:
        print("리뷰 탭 클릭 오류:", e)

    # 리뷰 총 개수 가져오기
    print("\n리뷰 총 개수 가져오기")
    try:
        review_count_element = WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/h2/div[1]/em'))
        )
        total_reviews = int(review_count_element.text.replace(',', ''))
        print("총 리뷰 수:", total_reviews)
    except Exception as e:
        print("리뷰 총 개수 가져오기 오류:", e)

    review_count = 0
    reviews = []

    while review_count < total_reviews:  # 원하는 수의 리뷰를 수집할 때까지 반복
        review_elements = driver.find_elements(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/ul/li')
        total_elements_on_page = len(review_elements)

        for i in range(review_count + 1, total_elements_on_page + 1):  # 올바른 인덱스를 보장
            try:
                more_button_xpath = f'//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/ul/li[{i}]/div[5]/a[2]'
                review_text_xpath = f'//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/ul/li[{i}]/div[5]/a[1]'

                # '더보기' 버튼이 있는 경우 클릭 후 텍스트 스크래핑
                try:
                    more_button = driver.find_element(By.XPATH, more_button_xpath)
                    if more_button.is_displayed():
                        driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", more_button)
                        driver.execute_script("arguments[0].click();", more_button)
                        print(f"리뷰 {review_count + 1}의 '더보기' 버튼 클릭 완료")
                        WebDriverWait(driver, 2).until(EC.presence_of_element_located((By.XPATH, review_text_xpath)))
                except NoSuchElementException:
                    print(f"리뷰 {review_count + 1}: '더보기' 버튼 없음, 바로 스크래핑 진행")
                except TimeoutException:
                    print(f"리뷰 {review_count + 1}: '더보기' 버튼 클릭 시간 초과")
                except ElementClickInterceptedException:
                    print("다른 요소가 클릭을 가로챔, 스크롤 조정 후 다시 시도")
                    driver.execute_script("window.scrollBy(0, 100);")  # 아주 작은 스크롤 조정
                    more_button.click()

                # 리뷰 텍스트 추출
                review_text_element = driver.find_element(By.XPATH, review_text_xpath)
                review_text = review_text_element.text.strip()

                if review_text:
                    reviews.append(review_text)
                    review_count += 1
                    print(f"리뷰 {review_count}: {review_text}\n")
                else:
                    print(f"리뷰 {review_count}: 내용 없음")
                    review_count += 1

            except (NoSuchElementException, TimeoutException, StaleElementReferenceException):
                print(f"리뷰 {review_count + 1}: 리뷰 텍스트 없음 또는 요소 갱신됨")
                continue

        if review_count < total_reviews:
            # 하단 '더보기' 버튼 클릭하여 다음 리뷰 로드
            try:
                bottom_more_button = WebDriverWait(driver, 5).until(
                    EC.element_to_be_clickable((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[2]/div/a'))
                )
                driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", bottom_more_button)
                bottom_more_button.click()
                WebDriverWait(driver, 5).until(
                    EC.presence_of_all_elements_located((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/ul/li'))
                )
                print("하단 '더보기' 버튼 클릭 완료")
            except (NoSuchElementException, TimeoutException, ElementClickInterceptedException) as e:
                print("하단 '더보기' 버튼을 찾을 수 없거나 클릭할 수 없음:", e)
                break

    print(f"총 {review_count}개의 리뷰를 가져왔습니다.")

    # CSV 파일로 저장
    reviews_df = pd.DataFrame(reviews, columns=['Review'])
    csv_filename = f"{store_name}_reviews.csv"
    reviews_df.to_csv(csv_filename, index=False, encoding='utf-8-sig')
    print(f"리뷰 데이터가 '{csv_filename}' 파일로 저장되었습니다.")

except Exception as e:
    print("오류 발생:", e)
